In [1]:
import structure as st
from tkinter import filedialog
import customtkinter as ctk
import re
import os
import ctypes
from ctypes import wintypes

In [15]:
def get_desktop_path():
    CSIDL_DESKTOP = 0x0000  # Desktop folder constant
    SHGFP_TYPE_CURRENT = 0
    buf = ctypes.create_unicode_buffer(wintypes.MAX_PATH)
    ctypes.windll.shell32.SHGetFolderPathW(None, CSIDL_DESKTOP, None, SHGFP_TYPE_CURRENT, buf)
    return buf.value

def read_input(cds):
    valid_characters=['A', 'G', 'C', 'T','U']
    if not set(cds) in valid_characters:
        cds=''.join([c for c in cds if c in valid_characters])
    seq_entry.delete('1.0',ctk.END)
    seq_entry.insert('1.0',cds)
    if radio_var.get()==0:
        cds_seq,gc_converted=change_codon()
    else:
        cds_seq,gc_converted=alternative_ccc()
    gc_initial = gc_content(cds)
    gc_label_initial.configure(text=f'GC content_iniot: {gc_initial}%')
    return cds_seq,gc_converted

def write_output(cds,gc_converted):
    result_entry.configure(state=ctk.NORMAL)
    result_entry.delete("1.0", ctk.END)
    result_entry.insert('1.0',cds.upper()) 
    result_entry.configure(state=ctk.DISABLED)
    gc_converted = gc_content(cds)
    gc_label_converted.configure(text=f'GC content: {gc_converted}%')
    
def open_file():
    file_path = filedialog.askopenfilename(filetypes=[("FASTA files", "*.fasta"), ("Text files", "*.txt")])
    cds=st.fasta_to_list(fasta_dir=file_path,seq_to_codon=False,sos_eos=False)[0].upper()
    if file_path:
        cds_seq,gc_converted=read_input(cds)
        write_output(cds_seq,gc_converted)
        codon_seperation()
            
def submit_sequence():
    sequence = seq_entry.get("1.0", ctk.END)
    cleaned_cds = sequence.strip().replace('\n','').upper()
    cds_seq,gc_converted=read_input(cleaned_cds)
    write_output(cds_seq,gc_converted)
    codon_seperation()

def codon_seperation():
    if check_var.get()==1:
        entry_sequence = seq_entry.get("1.0", ctk.END)
        seq_entry.delete('1.0',ctk.END)
        cds = st.seq_to_cds([entry_sequence],sos_eos=False,remove_stop=False)[0] 
        seq_entry.insert('1.0',cds)
 
        result_entry.configure(state=ctk.NORMAL)
        result_seq = result_entry.get("1.0", ctk.END)
        result_entry.delete('1.0',ctk.END)
        cds = st.seq_to_cds([result_seq],sos_eos=False,remove_stop=False)[0] 
        result_entry.insert('1.0',cds)
        result_entry.configure(state=ctk.DISABLED)
    else:
        entry_sequence = seq_entry.get("1.0", ctk.END)
        seq_entry.delete('1.0',ctk.END)
        cds = entry_sequence.replace(' ','')
        seq_entry.insert('1.0',cds)
        
        result_entry.configure(state=ctk.NORMAL)
        result_seq = result_entry.get("1.0", ctk.END)
        result_entry.delete('1.0',ctk.END)
        cds = result_seq.replace(' ','')
        result_entry.insert('1.0',cds)
        result_entry.configure(state=ctk.DISABLED)

def change_codon():
    sequence = seq_entry.get("1.0", ctk.END)
    seq_entry.delete('1.0',ctk.END)
    only_letters = re.sub(r'[^A-Za-z]', '', sequence)
    cds = only_letters.replace(' ','')
    seq_entry.insert('1.0',cds)
    codon_list={
 'UUA': 'CUC', 'UUG': 'CUC', 'CUA': 'CUC', 'CUG': 'CUC', 'CUU': 'CUC', 
 'TTA': 'CTC', 'TTG': 'CTC', 'CTA': 'CTC', 'CTG': 'CTC', 'CTT': 'CTC',
 'UUU': 'UUC', 'TTT': 'TTC',
 'AUA': 'AUC', 'AUU': 'AUC', 'ATA': 'ATC', 'ATT': 'ATC',
 'GUA': 'GUC', 'GUG': 'GUC', 'GUU': 'GUC', 'GTA': 'GTC', 'GTG': 'GTC', 'GTT': 'GTC',
 'UCA': 'UCC', 'UCU': 'UCC', 'UCG': 'UCC', 'AGC': 'UCC', 'AGU': 'UCC',
 'TCA': 'TCC', 'TCT': 'TCC', 'TCG': 'TCC', 'AGT': 'TCC',
 'CCA': 'CCC', 'CCG': 'CCC', 'CCU': 'CCC', 'CCT': 'CCC',
 'ACA': 'ACC', 'ACG': 'ACC', 'ACU': 'ACC', 'ACT': 'ACC',
 'GCA': 'GCC', 'GCG': 'GCC', 'GCU': 'GCC', 'GCT': 'GCC',
 'AGA': 'CGC', 'AGG': 'CGC', 'CGA': 'CGC', 'CGG': 'CGC', 'CGU': 'CGC', 'CGT': 'CGC',
 'GAU': 'GAC', 'GAT': 'GAC',
 'AAU': 'AAC', 'AAT': 'AAC',
 'UGU': 'UGC', 'TGT': 'TGC',
 'GGA': 'GGC', 'GGG': 'GGC', 'GGU': 'GGC', 'GGT': 'GGC',
 'CAU': 'CAC', 'CAT': 'CAC',
 'UAU': 'UAC', 'TAT': 'TAC',
 'GAA': 'GAG', 'CAA': 'CAG', 'AAA': 'AAG'
}
    rep = dict((re.escape(k), v) for k, v in codon_list.items()) 
    pattern = re.compile("|".join(rep.keys()))
    cds = st.seq_to_cds([cds],sos_eos=False,remove_stop=False)[0].upper()
    cds_seq = pattern.sub(lambda m: rep[re.escape(m.group(0))], cds).replace(' ','')  
    gc_converted = gc_content(cds_seq)
    return cds_seq,gc_converted     

def alternative_ccc():
    cds,gc_converted=change_codon()
    alternative_codon={'CUC':'CUG','CTC':'CTG',
                      'UUC':'UUU','TTC':'TTT',
                      'AUC':'AUA','ATC':'ATA',
                      'GUC':'GUG','GTC':'GTG',
                      'UCC':'UCG','TCC':'TCG',
                      'CCC':'CCG',
                      'ACC':'ACG',
                      'GCC':'GCG',
                      'CGC':'CGG',
                      'GAC':'GAT',
                      'AAC':'AAT',
                      'UGC':'UGU','TGC':'TGT',
                      'GGC':'GGG',
                      'CAC':'CAT',
                      'UAC':'UAU','TAC':'TAT',
                      'GAA':'GAG',
                      'CAA':'CAG',
                      'AAA':'AAG'} #some of them just works for T instead of U 
    lst=(st.seq_to_cds([cds],sos_eos=False,remove_stop=False)[0].upper()).split()
    for i,v in enumerate(lst[:-1]):
        if (v[-1]=='C'and v[1]!='C' and lst[i+1][0:2]=='CC') or (v[0]!='C'and v[1:]=='CC' and lst[i+1][0]=='C') or v=='CCC':
            lst[i]=alternative_codon[v]
            
    if lst[-1]=='CCC':
            lst[-1]='CCG'
            
    cds_seq=''.join(lst)
    gc_converted = gc_content(cds_seq)
    return cds_seq,gc_converted

def gc_content(cds):
    if len(cds.replace(' ',''))!=0:
        gc=round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds.replace(' ',''))*100,1)
    else:
        gc=0
    return gc 
    
ctk.set_appearance_mode('dark')
ctk.set_default_color_theme('blue')

root = ctk.CTk()
root.title("Wobble to C converter")
root.geometry('600x600')

root.rowconfigure((1,4),weight=4, uniform = 'a')
root.rowconfigure((2,5),weight=6, uniform = 'a')
root.rowconfigure((0,3,6,7),weight=3, uniform = 'a')
root.columnconfigure((0,1,2),weight=1, uniform = 'a')

file_button = ctk.CTkButton(root, text="Select FASTA or TEXT File", command=open_file)
file_button.grid(row = 0, column = 0, padx = 5, pady = 5, sticky = 'w')

seq_label = ctk.CTkLabel(root, text="Or enter CDS sequence below",text_color='gray')
seq_label.grid(row = 1, column = 0, padx = 5, pady = 5, sticky = 'sw')

# Checkbox
check_var=ctk.IntVar()
check_box=ctk.CTkCheckBox(root,text='seperation based on codon',variable=check_var,command=codon_seperation)
check_box.grid(row = 1, column =1, padx = 5, pady = 5, sticky = 'sw')# Checkbox

radio_var=ctk.IntVar()
# The change codon
radio_change_codon=ctk.CTkRadioButton(root,text='Change codon to codon with the most C',variable=radio_var,command=change_codon,value=0)
radio_change_codon.grid(row =1 , column =2, padx = 5, pady = 5, sticky = 'nw')

# No three Cs in a row
radio_var_frame=ctk.CTkRadioButton(root,text='No three Cs in a row',variable=radio_var,command=alternative_ccc,value=1)
radio_var_frame.grid(row =1 , column =2, padx = 5, pady = 5, sticky = 'sw')

# Input Textbox
seq_entry = ctk.CTkTextbox(root)
seq_entry.grid(row=2,column=0, sticky = 'nsew', padx = 5, pady = 5,columnspan=3)

#GC-content for initial codons
gc_label_initial=ctk.CTkLabel(root,text='')
gc_label_initial.grid(row=3,column=1, pady = 5, sticky = 'n')

# Submit button
submit_button = ctk.CTkButton(root, text="Submit", command=submit_sequence)
submit_button.grid(row = 3, column = 0, padx = 5, pady = 5, sticky = 'nw')

#GC-content for initial codons
Converted_seq_label=ctk.CTkLabel(root,text='Converted seq',text_color='gray')
Converted_seq_label.grid(row=4,column=0, pady = 5, sticky = 'sw')

# Result Textbox
result_entry = ctk.CTkTextbox(root)
result_entry.grid(row=5,column=0, sticky = 'nsew', padx = 5, pady = 5,columnspan=3)

# GC-content for converted codons
gc_label_converted=ctk.CTkLabel(root,text='')
gc_label_converted.grid(row=6,column=1, pady = 5, sticky = 'n')

# save label
save_label = ctk.CTkLabel(root,text='',text_color='gray')
save_label.grid(row=6,column=1,pady=10, sticky = 's')

# save label
developer_label = ctk.CTkLabel(root,text='barzegar@postech.ac.kr',text_color='gray')
developer_label.grid(row=7,column=1,pady=10, sticky = 's')

# Run the GUI
root.mainloop()


In [10]:
lst=['aaa','ccc']

In [11]:
lst[-1]

'ccc'